In [ ]:
# import packages
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso, LassoCV
from sklearn.datasets import make_regression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split


In [ ]:
######################################

In [ ]:
# toy data
X, Y = make_regression(n_samples=1000, n_features=25, noise=10, random_state=0)

In [ ]:
#######################################

# RANDOM FOREST

In [ ]:
# instantiate and fit
r_forest = RandomForestRegressor(max_depth=100, random_state=0)
r_forest.fit(X,Y)

In [ ]:
# predict
r_forest.predict(X[:,])

In [ ]:
####################################

# LASSO

In [ ]:
# instantiate lasso with 10-fold cross-validation
lasso = LassoCV(cv=10,random_state=0) # random_state for reproducible results
lasso_poly = LassoCV(cv=10, random_state=0)

In [ ]:
# fit
lasso.fit(X,Y)

In [ ]:
# compute polynomial features for lasso
poly = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)
X_poly = poly.fit_transform(X)

In [ ]:
# predict values
lasso.predict(X[:,]) # TODO: INPUT TEST SET HERE? OR APPROPRIATE DATA

In [ ]:
lasso_poly.fit(X_poly,Y) # here it takes 1m 25s for 25 original features with degree 3 poly transformation

In [ ]:
lasso_poly.predict(X_poly[:,]) # prediction is fast

In [ ]:
# Define the LASSO base-learner # TODO: CHANGE THIS!!!, PROBABLY MAKE A CHILDCLASS OF LASSOCV!!!
# DONT KNOW IF NEEDED LIKE THAT!
class Base_Lasso(LassoCV):
    pass
    """""   X: features.
            Y: labels.
            poly_degree: number of degrees for polynomial transformation.
            poly_reg: whether to fit the model to the polynomial transformed X or not. """""
    """def __init__(self, x, y, poly_degree=1, poly_reg=False):
        super().__init__(cv=10,random_state=0)
        self.x = x
        self.y = y
        self.poly_degree = poly_degree
        self.poly_reg = poly_reg
        self.x_poly = PolynomialFeatures(degree=self.poly_degree, interaction_only=False, include_bias=False).fit_transform(self.x) # TODO: OR SEPARATE FUNCTION? """

    #def X_poly(self): # TODO: MAYBE DELETE?
        #poly = PolynomialFeatures(degree=self.poly_degree, interaction_only=False, include_bias=False)
        #return poly.fit_transform(self.X)

    "Comment on this function"

    """def fit(self,cv=10,random_stat=0):
        self.lasso = LassoCV(cv=cv, random_state=random_stat)
        if self.poly_reg:
            self.lasso.fit(self.x_poly, self.y)
        else:
            self.lasso.fit(self.x, self.y)"""


    "Comment on this function"
    #def predict(self,x):
        #return lasso_reg.predict(x)

In [ ]:
lol = Base_Lasso(cv=10, random_state=0)

In [ ]:
lol.fit(X,Y)

In [ ]:
lol.predict(X[:,]) # oder so

In [ ]:
#####################################

# Neural Network

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


In [ ]:
# set some parameters
d = 25 #dimensions
N = 10000 #Number of observations

In [ ]:
x, y = make_regression(n_samples=N, n_features=d, noise=1, random_state=0)
# train test split (just as an example)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

Set up Model

In [ ]:
# make model
# 3 layers with 200 units (elu activation), 2 layers with 100 units (elu activations), 1 output layer (linear activation)
model = keras.Sequential([
    keras.Input(shape=(d,)),
    layers.Dense(units=200, activation="elu", name="layer1"),
    layers.Dense(units=200, activation="elu", name="layer2"),
    layers.Dense(units=200, activation="elu", name="layer3"),
    layers.Dense(units=100, activation="elu", name="layer4"),
    layers.Dense(units=100, activation="elu", name="layer5"),
    layers.Dense(units=1, activation="linear", name="layer6"),

], name="Dense_Neural_Network")
model.summary()

In [ ]:
x, y = make_regression(n_samples=N, n_features=d, noise=1, random_state=0)

In [ ]:
# train test split (just as an example)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

Compile Model

In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(),  # Optimizer
    # Loss function to minimize
    loss=keras.losses.MeanSquaredError(),
    # List of metrics to monitor
    metrics=[keras.metrics.MeanSquaredError()],
)

Training

In [ ]:
# Add early stopping
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=0, start_from_epoch=0)

In [ ]:
print("Training Model")
training = model.fit(
    X_train,
    y_train,
    batch_size=100,
    epochs=100,
    validation_data=(X_test, y_test),
    callbacks=[callback] # include early stopping
)

In [ ]:
training.history # to see training history

Evaluate

In [ ]:
print("Evaluate on test data")
results = model.evaluate(X_test, y_test, batch_size=100)
print("test loss, test acc:", results)

Predictions

In [ ]:
print("Generate predictions for test samples")
predictions = model.predict(X_test)
print("predictions shape:", predictions.shape)
predictions